# Useful Functions on DataFrames
Here are some of the most useful functions on DataFrames. 

**df.columns()** - Returns a list of column names. Column names can be used as keys to extract columns from a dataframe. <br>
**df.head(#)** and **df.tail(#)** - Returns the first (head) or last (tail) # rows of a dataframe. <br>
**df.describe()** - Returns quick statistics by column, including mean, median, <br>
**df.sort_values()** - Sort values in a column, similar to using a Filter in Excel. Note that the index remains the same. <br>
**df.sort_index()** - Sort by index. Useful for returning order after sorting by a certain column. <br>
**df.groupby()** - Collapse rows together that share one or more same column values. Useful for going from a fine to coarse analysis. For example, if data is saved by country, we could groupby country and analyze by region or continent. <br>
**df.explode()** - Using a list in some column's value, create new rows for each entry in that list. 
**df.apply()**

In [3]:
import numpy as np
import pandas as pd

# For this example, I downloaded May 2022 data
# This will take 1-2 minutes as there are 400,000+ rows of data!
df = pd.read_excel("../Datasets/all_data_M_2022.xlsx")

In [2]:
df.describe() # Quick statistics by column

NameError: name 'df' is not defined

In [ ]:
area_titles = sanitized_df['AREA_TITLE'].value_counts() # List of all unique entiries and their count in a column
#for text in area_titles:
#    print(text)
area_titles

In [ ]:
sanitized_df.sort_values(by='H_MEDIAN', inplace=True) # Sort by a column. Note the index remains
sanitized_df

In [ ]:
sanitized_df.sort_index() # Sort by index. Useful for returning order after sorting by a certain column